In [1]:
from thunder_QuartzNet import QuartzNet, conv1d_decoder
from loading_weights import load_quartznet_checkpoint, QuartznetCheckpoint

In [26]:
import torch

In [2]:
encoder, decoder = load_quartznet_checkpoint(QuartznetCheckpoint.QuartzNet15x5Base_En)

{'filters': [256, 256, 256, 256, 256, 256, 512, 512, 512, 512, 512, 512, 512, 512, 512], 'kernel_sizes': [33, 33, 33, 39, 39, 39, 51, 51, 51, 63, 63, 63, 75, 75, 75]} {'sample_rate': 16000, 'n_window_size': 320, 'n_window_stride': 160, 'n_fft': 512, 'nfilt': 64, 'dither': 1e-05}
[' ', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', "'"]


In [5]:
old_vocab = [' ', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', "'"]

new_vocab = {'': 0,
 'ʃ': 1,
 'i': 2,
 'ɦ': 3,
 'æ': 4,
 'd̚': 5,
 'd': 6,
 'j': 7,
 'ɝ': 8,
 'ɑ': 9,
 'r': 10,
 'k̚': 11,
 'k': 12,
 's': 13,
 'u': 14,
 'ɾ': 15,
 'ɪ': 16,
 'ŋ': 17,
 'ɡ̚': 18,
 'g': 19,
 'w': 20,
 'ʔ': 21,
 'ɔ': 22,
 'l': 23,
 'ɚ': 24,
 'oʊ': 25,
 'n': 26,
 'm': 27,
 't̚': 28,
 't': 29,
 'ɨ̞': 30,
 'ɛ': 31,
 'ɔɪ': 32,
 'aɪ': 33,
 'ð': 34,
 'h': 35,
 'z': 36,
 'p̚': 37,
 'ə': 38,
 'θ': 39,
 'b̚': 40,
 'b': 41,
 'ʉ': 42,
 'f': 43,
 'l̩': 44,
 'v': 45,
 'aʊ': 46,
 'p': 47,
 'ʌ': 48,
 'eɪ': 49,
 'n̩': 50,
 'tʃ': 51,
 'ʊ': 52,
 'dʒ': 53,
 'ɾ̃': 54,
 'ə̥': 55,
 'ʒ': 56,
 'm̩': 57,
 'ŋ̩': 58}

In [22]:
decoder.weight.shape

torch.Size([29, 1024, 1])

In [12]:
old_vocab = {k:v for (v,k) in enumerate(old_vocab)}
old_vocab[''] = 28

In [42]:
overlap = {k:k for k in (old_vocab.keys() & new_vocab.keys())}

In [46]:
firsts = {k:k[0] for k in new_vocab.keys() if k and k[0] in old_vocab.keys()}

In [47]:
firsts | overlap

{'i': 'i',
 'd̚': 'd',
 'd': 'd',
 'j': 'j',
 'r': 'r',
 'k̚': 'k',
 'k': 'k',
 's': 's',
 'u': 'u',
 'g': 'g',
 'w': 'w',
 'l': 'l',
 'oʊ': 'o',
 'n': 'n',
 'm': 'm',
 't̚': 't',
 't': 't',
 'aɪ': 'a',
 'h': 'h',
 'z': 'z',
 'p̚': 'p',
 'b̚': 'b',
 'b': 'b',
 'f': 'f',
 'l̩': 'l',
 'v': 'v',
 'aʊ': 'a',
 'p': 'p',
 'eɪ': 'e',
 'n̩': 'n',
 'tʃ': 't',
 'dʒ': 'd',
 'm̩': 'm',
 '': ''}

In [48]:
with torch.no_grad():
    new_decoder = conv1d_decoder(1024, len(new_vocab))
    weight = decoder.weight
    bias = decoder.bias
    for token in overlap:
        new_decoder.weight[new_vocab[token]] = weight[old_vocab[token]]
        new_decoder.bias[new_vocab[token]] = bias[old_vocab[token]]

In [37]:
weight[28]

tensor([[ 0.0338],
        [-0.0420],
        [-0.5932],
        ...,
        [ 0.0685],
        [ 0.0258],
        [ 0.0061]], grad_fn=<SelectBackward>)

In [38]:
new_decoder.weight[0]

tensor([[ 0.0338],
        [-0.0420],
        [-0.5932],
        ...,
        [ 0.0685],
        [ 0.0258],
        [ 0.0061]], grad_fn=<SelectBackward>)